### Middleware

middleware provides a way to more tightly control what happens inside the agent , middleware is useful for following:

* Tracking agent behaviour with logging , analytics and debugging
* Transforming prompts, tool selection, and output formatting
* Adding retries, fallbacks and early termination logic
* Applying rate limits.

In [2]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model =init_chat_model("groq:qwen/qwen3-32b")

### Summarization middleware

Automatically summarizes conversation history when approaching token limits, preserving recent messages while compressing older context. Summarization is useful for the following:

* Long-running conversation that exceed context windows
* Multi-turn dialogues with extensive history
* Applications where preserving full conversation context matters

In [3]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage,SystemMessage

### Messagebased Summarization
agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=model,
            trigger=("messages",10),
            keep=("messages",4) ## keep top 4 messages at top
        )
    ]
)

In [4]:
### Run with thread id
config = {"configurable":{"thread_id":"test-1"}} ## uniquely identify the user

In [5]:
questions = [
    "what is 2+2?",
    "what is 10*5",
    "what is 100/4",
    "what is 100*5",
    "what is 105/5",
    "what is 10%5",
]

In [6]:
for q in questions:
    reponse = agent.invoke({"messages":[HumanMessage(content=q)]},config=config)
    print(f"Messages: {reponse}")
    print(f"Messages: {len(reponse['messages'])}")

Messages: {'messages': [HumanMessage(content='what is 2+2?', additional_kwargs={}, response_metadata={}, id='fe40c11b-bbeb-40ea-989a-c4e3a859dc81'), AIMessage(content='<think>\nOkay, so the user is asking "what is 2+2?" Hmm, that seems pretty straightforward. Let me think. In arithmetic, when you add two and two, the result is four. But wait, maybe there\'s a trick here? Like, sometimes in math problems, people use different number systems or maybe it\'s a riddle. Let me check. If it\'s base 10, which is standard, 2+2 is definitely 4. If it\'s in another base, like base 3, then 2+2 would be 11, but the question doesn\'t specify a different base. Also, in some contexts, like if you\'re talking about objects, like two apples plus two apples is four apples, but maybe in a different context? For example, in modular arithmetic, but again, unless specified, it\'s safe to assume standard addition. I don\'t think the user is trying to trick me here. They probably just want the basic answer. Le

### Token Size

In [7]:
from langchain.tools import tool
from langchain_core.messages import HumanMessage

@tool
def search_hotels(city:str)->str:
    """Search hotels - returns long responce to use more tokens."""
    return f"""Hotels in {city}:
    1. Grand Hotel - 5 star ,350rs/night,spa,pool,gym
    2. City Inn - 4 star, 180rs/night,business center
    3. Budget Stay - 3 star. 75rs/night, free wifi"""

agent = create_agent(
    model=model,
    tools=[search_hotels],
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=model,
            trigger= ("tokens",550),
            keep=("tokens",200),

        )
    ]
)

config = {"configurable":{"thread_id":"test-2"}}

def count_tokens(messages):
    total_char = sum(len(str(m.content)) for m in messages)
    return total_char//4  ## 4 chars = 1 token

In [8]:
cities = ["paris","london","delhi","dubai","Hyderabad","singapore"]

for city in cities:
    response = agent.invoke(
        {"messages":[HumanMessage(content=f"find hotels in {city}")]},
        config=config
    )

    tokens = count_tokens(response["messages"])
    print(f"{city}: ~{tokens} tokens,{len(response['messages'])} messages")
    print(f"{(response['messages'])}")

paris: ~151 tokens,4 messages
[HumanMessage(content='find hotels in paris', additional_kwargs={}, response_metadata={}, id='04fe2f44-e5b9-4abc-b6c8-f90ea700ab42'), AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user wants to find hotels in Paris. Let me check the available tools. There\'s a function called search_hotels that takes a city parameter. The city here is Paris. I need to make sure the parameters are correctly formatted. The function requires the city as a string. So I should call search_hotels with {"city": "Paris"}. I\'ll generate the tool call accordingly.\n', 'tool_calls': [{'id': 'bgkne47jp', 'function': {'arguments': '{"city":"Paris"}', 'name': 'search_hotels'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 156, 'total_tokens': 262, 'completion_time': 0.162471135, 'completion_tokens_details': {'reasoning_tokens': 81}, 'prompt_time': 0.006293152, 'prompt_tokens_details': None, 'queue_time': 

### Human In Loop MiddleWare

Pauses execution for the human approval, editing or rejection of tool calls before they execute. Human in loop is useful for the following:

* High stakes operations requiring human approval
* compilance workflows where human oversight is mandatory
* long running conversations where human feedback guides the agent

In [9]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

def read_email_tool(email_id:str)->str:
    """Mock function to read an email by its ID."""
    return f"Email content for ID:{email_id}"

def send_email_tool(recipient:str,subject:str,body:str)->str:
    """Mock function to send an email"""
    return f"Email sent tp {recipient} with subject '{subject}'"

In [11]:
agent = create_agent(
    model= model,
    tools= [read_email_tool,send_email_tool],
    checkpointer=InMemorySaver(),
    middleware={
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool":{
                    "allowed_decisions":["approve","edit","reject"]
                },
                "read_email_tool":False
            }
        )
    }
)

In [12]:
config = {"configurable":{"thread_id":"test-approve"}}

result = agent.invoke(
    {"messages":[HumanMessage(content="send email to john@test.com with subject 'Hello' and body 'How are you'?")]},
    config=config
)

In [13]:
result

{'messages': [HumanMessage(content="send email to john@test.com with subject 'Hello' and body 'How are you'?", additional_kwargs={}, response_metadata={}, id='1e822b5e-0bc0-4e2c-8f23-21f98004daf1'),
  AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user wants to send an email to john@test.com with the subject 'Hello' and body 'How are you'. Let me check the available tools. There's the send_email_tool which requires recipient, subject, and body. All three are required. The parameters are all strings. The user provided all three: recipient is john@test.com, subject is 'Hello', and body is 'How are you'. So I need to call the send_email_tool with these arguments. No issues here, all required parameters are present. Just format the JSON correctly.\n", 'tool_calls': [{'id': 'xwjtdhgqc', 'function': {'arguments': '{"body":"How are you?","recipient":"john@test.com","subject":"Hello"}', 'name': 'send_email_tool'}, 'type': 'function'}]}, response_metadata={'token_usage

In [14]:
from langgraph.types import Command
if "__interrupt__" in result:
    print("Pasued Approving...")

    result = agent.invoke(
        Command(
            resume={
                "decisions":[
                    {"type":"approve"}
                ]
            }
        ),
        config=config
    )

    print(f"Result: {result['messages'][-1].content}")

Pasued Approving...
Result: The email has been successfully sent to john@test.com with the subject "Hello". Let me know if you need anything else!


In [15]:
result

{'messages': [HumanMessage(content="send email to john@test.com with subject 'Hello' and body 'How are you'?", additional_kwargs={}, response_metadata={}, id='1e822b5e-0bc0-4e2c-8f23-21f98004daf1'),
  AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user wants to send an email to john@test.com with the subject 'Hello' and body 'How are you'. Let me check the available tools. There's the send_email_tool which requires recipient, subject, and body. All three are required. The parameters are all strings. The user provided all three: recipient is john@test.com, subject is 'Hello', and body is 'How are you'. So I need to call the send_email_tool with these arguments. No issues here, all required parameters are present. Just format the JSON correctly.\n", 'tool_calls': [{'id': 'xwjtdhgqc', 'function': {'arguments': '{"body":"How are you?","recipient":"john@test.com","subject":"Hello"}', 'name': 'send_email_tool'}, 'type': 'function'}]}, response_metadata={'token_usage

##### Similarly for Reject

In [28]:
config = {"configurable":{"thread_id":"test-reject2"}}

result = agent.invoke(
    {"messages":[HumanMessage(content="send email to john@test.com with subject 'Hello' and body 'How are you'?")]},
    config=config
)

In [29]:
result

{'messages': [HumanMessage(content="send email to john@test.com with subject 'Hello' and body 'How are you'?", additional_kwargs={}, response_metadata={}, id='fa50a3c3-af13-4b98-856b-9aafd8c9a2e5'),
  AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user wants to send an email to john@test.com with the subject 'Hello' and body 'How are you?'. Let me check the available tools. There's the send_email_tool which requires recipient, subject, and body. All three are required. The parameters are all strings. The recipient is provided as john@test.com, subject is 'Hello', and body is 'How are you?'. I need to structure the function call with these parameters. Let me make sure I didn't miss any required fields. Yep, all three are there. So the tool_call should be send_email_tool with those arguments.\n", 'tool_calls': [{'id': '7dtt9rxtf', 'function': {'arguments': '{"body":"How are you?","recipient":"john@test.com","subject":"Hello"}', 'name': 'send_email_tool'}, 'type'

In [30]:
from langgraph.types import Command
if "__interrupt__" in result:
    print("Paused approving...")

    result = agent.invoke(
        Command(
            resume={
                "decisions":[
                    {"type":"reject"}
                ]
            }
        ),
        config=config
    )

    print(f"Result: {result['messages'][-1].content}")

Paused approving...
Result: It looks like your request to send an email to `john@test.com` with the subject "Hello" and body "How are you?" was rejected by the system (rejection ID: `7dtt9rxtf`). This could happen for reasons like invalid formatting, policy violations, or system limitations. 

Would you like to:
1. Try adjusting the email content or recipient?
2. Check if there are additional constraints for the mock email tool?
3. Attempt a different action?

Let me know how you'd like to proceed!


In [31]:
result

{'messages': [HumanMessage(content="send email to john@test.com with subject 'Hello' and body 'How are you'?", additional_kwargs={}, response_metadata={}, id='fa50a3c3-af13-4b98-856b-9aafd8c9a2e5'),
  AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user wants to send an email to john@test.com with the subject 'Hello' and body 'How are you?'. Let me check the available tools. There's the send_email_tool which requires recipient, subject, and body. All three are required. The parameters are all strings. The recipient is provided as john@test.com, subject is 'Hello', and body is 'How are you?'. I need to structure the function call with these parameters. Let me make sure I didn't miss any required fields. Yep, all three are there. So the tool_call should be send_email_tool with those arguments.\n", 'tool_calls': [{'id': '7dtt9rxtf', 'function': {'arguments': '{"body":"How are you?","recipient":"john@test.com","subject":"Hello"}', 'name': 'send_email_tool'}, 'type'